<a href="https://colab.research.google.com/github/vtponciano-ufrj/Grafos/blob/main/novo_correlecion_claster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

!apt-get install -y -qq glpk-utils
!pip install pyomo
import signal
import time
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import numpy as np

m = pyo.ConcreteModel()

A = (np.array([[ 0, 0, -1, 0, 0, 0, 0, 1, 0, -1, 0, 0, 1, 1, 0, -1, 1],
[0, 0, 0, 1, 1, 0, 0, 0, 0, -1, 0, 1, 0, 0, -1, -1, 1],
[0, 0, 0, 0, 0, 1, 0, 0, 1, -1, 0, 1, 0, -1, 0, -1, 1],
[0, 1, -1, 0, 1, 1, 0, 0, 0, -1, 0, 0,  0, -1, 0, 0, 1],
[0, 1, 0, 1, 0, 0, 0, 0, 1, -1, 0, 0, 1, 0, -1, -1, 0],
[0, 0, -1, 1, 0, 0, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1, 0],   
[0, 1, 0, 0, -1, 0, 0, 0, 0, -1, 1, 1, 0, 0, 0, -1, 1],
[1, 0, -1, 1, 0, 1, 0, 0, 0, -1, 0, 0, 1, 0, 0, -1, 0],
[0 ,0, 0, 1, 0 ,1, 0, 0, 0, -1, 0, 0, 0, 1 ,-1, -1, 1],
[1, 0, -1, 0, 0, 0, 1, 1, 0, 0, 0, 1, -1, 0, 0, -1, 0],
[0, 1, 0, 1, 0, 0, 0, -1, 0, -1, 0, 1, 0, 0, 0, -1, 1],
[0, 1, 0, 0, 0, -1, 1, 0, 1, -1, 0, 0, 0,  0, -1, 0, 1],
[1, 0, -1 ,0 ,0 ,0, 1, 1, 0, -1, 0, 0, 0,0, 0, -1, 1],
[1 ,0, -1, 0, -1, 1, 0, 0, 1, -1, 1, 0, 0,  0, 0, 0, 0],
[0, 0, 0, 1, -1, 0, 0, 0, 1, -1, -1, 0, 0, 0, 0, 1, 1],
[0, 0, -1 ,1, 0, 0, 0, -1, 1, -1, 0, 1, 0, 0, 0, 0, 1],
[1, 1, -1, 1, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, 0, -1, 0 ]]))
 
P=( np.array([ [0, 0, 0, 0, 0 ,0 ,0 ,1, 0, 0, 0, 0, 1, 1,0, 0, 1 ],
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1 ],
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0 ,1 ],
[0 ,1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 1],
[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0 ],
[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0 ], 
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0 ],
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1 ],
[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0 ,1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
[0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,  0, 0, 0, 1 ],
[ 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1 ],
[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,  0, 0, 0, 0  ],
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1 ],
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1 ],
[1, 1, 0, 1, 0,0 ,0, 0, 1, 0,0, 0,0, 0, 0, 0, 0 ]]))



N = A - P

n = 17
K = 2




# Variáveis de decisão
m.x = pyo.Var(range(K), range(n), domain=pyo.Binary)
m.s = pyo.Var(range(K), range(K), domain=pyo.Binary)
m.d = pyo.Var(range(K), range(K), domain=pyo.NonNegativeReals)
m.z = pyo.Var(range(K), range(n), range(K), range(n), domain=pyo.Binary)

# Restrições
m.cons1 = pyo.ConstraintList()
for i in range(n):
    m.cons1.add(sum(m.x[k, i] for k in range(K)) == 1)

m.cons2 = pyo.ConstraintList()
for k in range(K):
    m.cons2.add(sum(m.x[k, i] for i in range(n)) >= 1)

m.cons3 = pyo.ConstraintList()
for k in range(K):
    for w in range(k, K):
        for i in range(n):
            for j in range(n):
                if i < j:
                    m.cons3.add(m.x[k, i] + m.x[w, j] - m.z[k, i, w, j] <= 1)
                    m.cons3.add(m.x[k, j] + m.x[w, i] - m.z[k, i, w, j] <= 1)

m.cons4 = pyo.ConstraintList()
for k in range(K):
    for w in range(k, K):
        m.cons4.add(m.d[k, w] <= sum(P[i, j] * m.z[k, i, w, j] for i in range(n) for j in range(n)) +
                    sum(P[i, j] * m.z[k, j, w, i] for i in range(n) for j in range(n)))
        m.cons4.add(m.d[k, w] <= sum(abs(N[i, j]) * m.z[k, i, w, j] for i in range(n) for j in range(n)) +
                    sum(abs(N[i, j]) * m.z[k, j, w, i] for i in range(n) for j in range(n)))
        m.cons4.add(m.d[k, w] - sum(P[i, j] * m.z[k, i, w, j] for i in range(n) for j in range(n)) -
                    sum(P[i, j] * m.z[k, j, w, i] for i in range(n) for j in range(n)) - np.count_nonzero(P) * m.s[k, w] >= -np.count_nonzero(P))
        m.cons4.add(m.d[k, w] - sum(abs(N[i, j]) * m.z[k, i, w, j] for i in range(n) for j in range(n)) -
                    sum(abs(N[i, j]) * m.z[k, j, w, i] for i in range(n) for j in range(n)) -
                    np.count_nonzero(N) * (1 - m.s[k, w]) >= -np.count_nonzero(N))

# Função objetivo
m.obj = pyo.Objective(expr=sum(m.d[k, w] for k in range(K) for w in range(k, K)), sense=pyo.minimize)



# Cria uma instância do solver GLPK e define o tempo limite de execução para 60 segundos
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
opt.options['tmlim'] = 60
# Resolve o modelo usando o solver GLPK com o tempo limite definido
result = opt.solve(m)

# Imprime o valor da função objetivo
print(result)






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Problem: 
- Name: unknown
  Lower bound: 21.0
  Upper bound: 21.0
  Number of objectives: 1
  Number of constraints: 848
  Number of variables: 707
  Number of nonzeros: 3675
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 36365
      Number of created subproblems: 36365
  Error rc: 0
  Time: 53.33211398124695
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

